### Introduction
Here we examine the Chinook.db from the digital record store with tables for artists, albums, media tracks, invoices and customers. There are 11 tables with over 15,000 lines of data.

We will complete a business analysis to see what insights we can uncover utilizing SQL queries.

![chinook-schema](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

In [1]:
!conda install -yc conda-forge ipython-sql

Solving environment: ...working... done

# All requested packages already installed.



In [2]:

%%capture
%load_ext sql
%sql sqlite:///chinook-unmodified.db

In [3]:

%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook-unmodified.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Selecting New Albums to Purchase
The Chinook record store has just signed a deal with a new record label.

#### Task: select the first three albums that will be added to the store

From a list of four with genre
* Red Tone (Punk)
* Slim Jim Bites (Blues)
* Meteor and the Girls (Pop)
* Slim Jim Bite (Blues)

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

Note: calculate sum from total and group by genre is not correct since one total can have many genre, make the sum is double many time. As SUM(cc.quantity) is int, must change to float then it can devide

In [4]:
%%sql

WITH customer_country AS 
    (
    SELECT 
        il.track_id,
        il.quantity,
        i.total,
        c.country
    FROM invoice_line il
    INNER JOIN invoice i ON il.invoice_id=i.invoice_id
    INNER JOIN customer c ON c.customer_id=i.customer_id
    WHERE c.country="USA"
    )

SELECT
    cc.country,
    g.name genre,
    SUM(cc.quantity) pruchased_quantity,
    CAST(SUM(cc.quantity)AS FLOAT)/(SELECT COUNT(*) from customer_country)*100  percentage
FROM track tr
INNER JOIN customer_country cc ON cc.track_id=tr.track_id
INNER JOIN genre g ON g.genre_id=tr.genre_id
GROUP BY 2
ORDER BY 3 DESC;

 * sqlite:///chinook-unmodified.db
Done.


country,genre,pruchased_quantity,percentage
USA,Rock,561,53.37773549000951
USA,Alternative & Punk,130,12.369172216936251
USA,Metal,124,11.798287345385347
USA,R&B/Soul,53,5.042816365366318
USA,Blues,36,3.425309229305423
USA,Alternative,35,3.3301617507136063
USA,Pop,22,2.093244529019981
USA,Latin,22,2.093244529019981
USA,Hip Hop/Rap,20,1.9029495718363463
USA,Jazz,14,1.3320647002854424


Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

* Red Tone (Punk)
* Slim Jim Bites (Blues)
* Meteor and the Girls (Pop)

It's worth keeping in mind that combined, these three genres only make up only 17% of total sales, so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.

### Analyzing Employee Sales Performance
Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. 
#### Task: analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [5]:
%%sql
WITH employee_value AS
    (
    SELECT 
        e.first_name||" "||e.last_name employee_name,
        e.hire_date,
        c.customer_id,
        i.total
    FROM employee e
    LEFT JOIN customer c ON e.employee_id=c.support_rep_id
    LEFT JOIN invoice i ON i.customer_id=c.customer_id
    )

SELECT 
    employee_name,
    hire_date,
    COUNT(DISTINCT customer_id) number_customer_care,
    ROUND(SUM(total),2) total_revenuge,
    ROUND(SUM(total)/COUNT(DISTINCT customer_id),2) avg_revenue_per_customer
FROM employee_value
GROUP BY 1
ORDER BY 3 DESC;

 * sqlite:///chinook-unmodified.db
Done.


employee_name,hire_date,number_customer_care,total_revenuge,avg_revenue_per_customer
Jane Peacock,2017-04-01 00:00:00,21,1731.51,82.45
Margaret Park,2017-05-03 00:00:00,20,1584.0,79.2
Steve Johnson,2017-10-17 00:00:00,18,1393.92,77.44
Robert King,2017-01-02 00:00:00,0,None,None
Nancy Edwards,2016-05-01 00:00:00,0,None,None
Michael Mitchell,2016-10-17 00:00:00,0,None,None
Laura Callahan,2017-03-04 00:00:00,0,None,None
Andrew Adams,2016-08-14 00:00:00,0,None,None


Looking at table, only Jane, Margaret and Steve take care customers. While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

### Analyzing Sales by Country
For each country, include:
* total number of customers
* total value of sales
* average value of sales per customer
* average order value


In [16]:
%%sql

WITH country_revenue AS
    (
    SELECT 
        c.country,
        c.customer_id,
        i.total
    FROM customer c
    INNER JOIN invoice i ON i.customer_id=c.customer_id
    )

SELECT 
    country, 
    COUNT(DISTINCT customer_id) as total_customers,
    ROUND(SUM(total),2) as total_sales,
    ROUND(SUM(total)/COUNT(DISTINCT customer_id),2) as avg_sale_per_customer,
    ROUND(AVG(total),2) as avg_order_value
FROM country_revenue
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///chinook-unmodified.db
Done.


country,total_customers,total_sales,avg_sale_per_customer,avg_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
France,5,389.07,77.81,7.78
Brazil,5,427.68,85.54,7.01
Germany,4,334.62,83.66,8.16
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Czech Republic,2,273.24,136.62,9.11
Sweden,1,75.24,75.24,7.52


Because there are a number of countries with only one customer, you should group these customers as "Other" in your analysis. You can use the following 'trick' to force the ordering of "Other" to last in your analysis.Based on the data, there may be opportunity in the following countries:

* Czech Republic
* United Kingdom
* India
It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

### Albums vs Individual Tracks
The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:
* purchase a whole album
* purchase a collection of one or more individual tracks.

The strategy: the management is considering to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

#### Task: find out what percentage of purchases are individual tracks vs whole albums

So that management can use this data to understand the effect this decision might have on overall revenue.


##### Some cases need to be considered:
* Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
* Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In [7]:
%%sql
DROP VIEW IF EXISTS number_tracks_per_invoice;
DROP VIEW IF EXISTS number_tracks_per_album;
DROP VIEW IF EXISTS invoice_album_track;
CREATE VIEW number_tracks_per_invoice AS
    SELECT 
        il.invoice_id, 
        al.album_id as album_id,
        COUNT(tr.track_id) as number_tracks
    FROM invoice_line il
    INNER JOIN track tr ON il.track_id=tr.track_id
    INNER JOIN album al ON al.album_id=tr.album_id
    GROUP BY 1,2;

CREATE VIEW number_tracks_per_album AS
    SELECT
        al.album_id,
        COUNT(tr.track_id) as number_tracks_per_album
    FROM album al
    INNER JOIN track tr ON tr.album_id=al.album_id
    GROUP BY 1;

CREATE VIEW invoice_album_track AS
SELECT 
    pv.invoice_id,
    pv.album_id,
    pv.number_tracks,
    pa.number_tracks_per_album,
    CASE
        WHEN pv.number_tracks= pa.number_tracks_per_album then "album"
        ELSE "individual_track"
        END AS album_individualtrack
FROM number_tracks_per_invoice as pv
LEFT JOIN number_tracks_per_album as pa ON pv.album_id=pa.album_id;

 * sqlite:///chinook-unmodified.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

From above table found that client can select some tracks from several album, some invoices have both individual track and album.So in order to have separate value of album & individual track, we caculate revenue of album first.

In [17]:
%%sql
select *
from invoice_album_track
limit 20;

 * sqlite:///chinook-unmodified.db
Done.


invoice_id,album_id,number_tracks,number_tracks_per_album,album_individualtrack
1,91,16,16,album
2,20,1,11,individual_track
2,34,1,17,individual_track
2,39,1,21,individual_track
2,66,1,10,individual_track
2,73,1,30,individual_track
2,134,1,10,individual_track
2,190,1,16,individual_track
2,215,1,14,individual_track
2,218,1,15,individual_track


##### caculate price of a album, then calculate revenue for individual and revenue

In [9]:
%%sql
DROP VIEW IF EXISTS album_price;
CREATE VIEW album_price AS 
SELECT 
    album_id,
    sum(unit_price) price
FROM 
    (
    SELECT 
        DISTINCT tr.track_id,
        tr.album_id, 
        il.unit_price
    FROM track tr
    INNER JOIN invoice_line il ON il.track_id=tr.track_id
    )
GROUP BY 1;

SELECT 
    SUM(alp.price) as sum_album,
    ((SELECT SUM(total) FROM invoice)- SUM(alp.price)) as sum_individual
FROM invoice_album_track iat
INNER JOIN album_price alp ON alp.album_id=iat.album_id
WHERE album_individualtrack="album"
GROUP BY album_individualtrack;

 * sqlite:///chinook-unmodified.db
Done.
Done.
Done.


sum_album,sum_individual
1553.3100000000009,3156.119999999993


##### second way: found that unit price is the same at 0.99 USD per track so we can calculate as below:

In [10]:
%%sql
SELECT
    album_individualtrack,
    COUNT (album_individualtrack) as purchased_number,
    SUM(number_tracks*0.99) as revenue
FROM invoice_album_track
GROUP BY album_individualtrack;

 * sqlite:///chinook-unmodified.db
Done.


album_individualtrack,purchased_number,revenue
album,207,1553.3100000000009
individual_track,2870,3156.1199999998175


Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.